# Ingesting realtime tweets using Apache Kafka, Tweepy and Python

## Purpose:

Main data source for the lambda architecture pipeline
uses twitter streaming API to simulate new events coming in every minute
Kafka Producer sends the tweets as records to the Kafka Broker
Contents:

- Twitter setup
- Defining the Kafka producer
- Producing and sending records to the Kafka Broker
- Deployment

In [ ]:
!pip3 install pyspark tweepy pymongo kafka-python "pymongo[srv]"

In [1]:
import tweepy
from tweepy import StreamingClient, StreamRule
from kafka import KafkaProducer
from pyspark.sql import functions as f
from datetime import datetime
from pyspark.sql import SparkSession
from kafka import KafkaProducer
import time
from datetime import datetime
import json
import pymongo
import os

## Base de données SQL

#  Configuration de Twitter 

getting the API object using authorization information

In [2]:
# twitter setup
consumer_key = "886bWUB38AHD1VC8vE777rVKs"
consumer_secret = "QLTWRcxbmxjOAAJatf4WCbL7j5vQYiyhSImv00wLarPVctcXE4"
access_token = "765095367067262976-Nz1XFSRSQjdd2MKYdPLiyKpjTUSsEoo"
access_token_secret = "8GZJcQVFJEk4SBrGgjk0V1HXiFau920Jt62Dntgf65qug"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAF0CeAEAAAAAep52nhowJZuk9B5RzizY8pJ7UfU%3Da5MK3zCKaEvktiqRgFrwwIt3gqkU53nNYWgnkl9bXbf2ygfscy"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object by passing in auth information
api = tweepy.API(auth) 

In [3]:
#try connection
try:
    api.verify_credentials()
    print("Authentication Successful")
except:
    print("Authentication Error")

Authentication Successful


# Création du producer Kafka 

- specify the Kafka Broker
- specify the topic name
- optional: specify partitioning strategy

In [4]:
topic_name = 'tweets_stream'

In [5]:
def kafka_stream_tweets(data):
    producer = KafkaProducer(
        value_serializer=lambda m: json.dumps(m).encode('utf-8'),
        bootstrap_servers='localhost:9092'
    )
    print(data)
    producer.send(topic_name, value=data)
    
    return True

In [6]:
class Streamer(StreamingClient):
    def on_data(self, data):
        json_data = json.loads(data)
        stream_messages = {}
        print(f"Preparing message ")
        event_time = datetime.now()
        stream_messages["tweet"] = json_data["data"]
        stream_messages["date_time"] = event_time.strftime("%Y-%m-%d %H:%M:%S")
        print(stream_messages)
        print(f"Kafka messages: {stream_messages}")
        kafka_stream_tweets(stream_messages)
        time.sleep(1)
        
        return True

    def on_connection_error(self):
        self.disconnect()
    
    def on_status(self, status):
        print(status.id)


KafkaPrinter = Streamer(bearer_token)
rule = StreamRule(value="harcelement")
KafkaPrinter.add_rules(rule)
KafkaPrinter.filter(tweet_fields="created_at")
KafkaPrinter.filter()

Stream encountered an exception
Traceback (most recent call last):
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tweepy/streaming.py", line 91, in _connect
    self.on_data(line)
  File "/var/folders/65/rsl7lrd91h178dgdgmq6ks9r0000gn/T/ipykernel_41767/874197983.py", line 11, in on_data
    kafka_stream_tweets(stream_messages)
  File "/var/folders/65/rsl7lrd91h178dgdgmq6ks9r0000gn/T/ipykernel_41767/1847031834.py", line 4, in kafka_stream_tweets
    bootstrap_servers='localhost:9092'
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/producer/kafka.py", line 383, in __init__
    **self.config)
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/client_async.py", line 244, in __init__
    self.config['api_version'] = self.check_version(timeout=check_timeout)
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/client_async.py", line 900, in check_version
    raise Errors.NoBrokersAv

Preparing message 
{'tweet': {'created_at': '2022-06-27T17:53:38.000Z', 'id': '1541479885612482562', 'text': 'RT @humanite_fr: Procès #FranceTelecom « Le PDG était parfaitement informé des remontées et alertes émises »\nLe parquet a requis des peines…'}, 'date_time': '2022-06-27 19:53:44'}
Kafka messages: {'tweet': {'created_at': '2022-06-27T17:53:38.000Z', 'id': '1541479885612482562', 'text': 'RT @humanite_fr: Procès #FranceTelecom « Le PDG était parfaitement informé des remontées et alertes émises »\nLe parquet a requis des peines…'}, 'date_time': '2022-06-27 19:53:44'}


Stream encountered an exception
Traceback (most recent call last):
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/tweepy/streaming.py", line 91, in _connect
    self.on_data(line)
  File "/var/folders/65/rsl7lrd91h178dgdgmq6ks9r0000gn/T/ipykernel_41767/874197983.py", line 11, in on_data
    kafka_stream_tweets(stream_messages)
  File "/var/folders/65/rsl7lrd91h178dgdgmq6ks9r0000gn/T/ipykernel_41767/1847031834.py", line 4, in kafka_stream_tweets
    bootstrap_servers='localhost:9092'
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/producer/kafka.py", line 383, in __init__
    **self.config)
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/client_async.py", line 244, in __init__
    self.config['api_version'] = self.check_version(timeout=check_timeout)
  File "/Users/svngoku/opt/anaconda3/envs/tf/lib/python3.7/site-packages/kafka/client_async.py", line 900, in check_version
    raise Errors.NoBrokersAv

Preparing message 
{'tweet': {'id': '1541480340971261952', 'text': 'RT @humanite_fr: Procès #FranceTelecom « Le PDG était parfaitement informé des remontées et alertes émises »\nLe parquet a requis des peines…'}, 'date_time': '2022-06-27 19:55:32'}
Kafka messages: {'tweet': {'id': '1541480340971261952', 'text': 'RT @humanite_fr: Procès #FranceTelecom « Le PDG était parfaitement informé des remontées et alertes émises »\nLe parquet a requis des peines…'}, 'date_time': '2022-06-27 19:55:32'}
